## 3장 Pandas로 데이터 처리하기
### pandas의 특징
- 쉬운 결손값 처리
- 레이블 위치를 자동적, 명시적으로 정리한 데이터 작성
- 데이터 집약
- 고도의 레이블 베이스의 슬라이싱, 추출, 큰 데이터셋트의 서브세트와
- 직감적인 데이터셋트 결합
- 데이터셋트의 유연한 변환 및 변형
- 축의 계층적 레이블 붙임
- 여러가지 데이터 형식에 대한 강력한 I/O
- 시계열 데이터 고유의 처리


### 샘플 데이터

https://github.com/practical-jupyter/sample-data

https://www.kaggle.com/CooperUnion/anime-recommendations-database


샘플 데이터 일람

- anime.csv : myanimelist.net의 정보를 수집한 원래 데이터
- anime_master.csv : anime.csv에서 형태 변환을 시랭한 결손 값을 제외한 데이터
- anime_split_genre.csv : anime_master.csv를 장르별로 분할한 데이터
- anime_genre_top10.csv : anime_split_genre.csv에서 멈버 수 상위 10 장르를 추출한 데이터
- anime_genre_top10_pivoted.csv : anime_genre_top10.csv의 장르를 새로축, 배급타입을 가로축으로 해서 멤버수의 합계로 크로스 집계한 데이터

### 샘플 데이터 경로

import os
base_url = 'https://raw.githubusercontent.com/practical-jupyter/sample-data/master/anime/'
anime_csv = os.path.join(base_url, 'anime.csv')

print(anime_csv)


    

In [1]:
import os
base_url = 'https://raw.githubusercontent.com/practical-jupyter/sample-data/master/anime/'
anime_csv = os.path.join(base_url, 'anime.csv')

print(anime_csv)


https://raw.githubusercontent.com/practical-jupyter/sample-data/master/anime/anime.csv


In [2]:
# 로컬 파일 패스 지정(다운로드한 장소에 따라 변경 필요)
anime_csv = './anime/anime.csv'
print(anime_csv)

./anime/anime.csv


### anime.csv 파일
anime.csv 에는 다음과 같은 데이터가 들어 있다. 
- anime_id : myanimelist.net에 등록된 작품 ID
- name : 작품의 이름
- genre : 작품의 장르
- type : 작품의 배급 타입
- episodes : 작품의 에피소드 수(영화는 1이 된다.)
- rating : myanimelist.net회원이 채점한 작품에 대한 평점(만점 10점)의 평균값
- members : 작품의 커뮤니티에 등록된 멤버 수 

In [3]:
import pandas as pd

anime_csv = './anime/anime.csv'
pd.read_csv(anime_csv).head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


### anime_master.csv 파일

anime_master.csv 는 anime.csv 를 가공한 데이터이다. 일례로 'Gintama&amp;#039' 가 'Gintama'로 바뀌어 있다. 


In [4]:
anime_master_csv = './anime/anime_master.csv'
pd.read_csv(anime_master_csv).head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama',"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


### anime_split_genre.csv 파일

anime_split_genre.csv 는 anime_master.csv 을 가공한 데이터이다. 복수의 장르가 하나의 행으로 정리되어 있는데 이것을 분할한다. 
장르마다 데이터를 집계할 경우에 사용한다. 

In [5]:
anime_split_genre_csv = './anime/anime_split_genre.csv'
pd.read_csv(anime_split_genre_csv).head()

,anime_id,name,genre,type,episodes,rating,members
0,20707,"""0""",Music,Music,1,5.06,1170
1,25627,"""Aesop"" no Ohanashi yori: Ushi to Kaeru, Yokub...",Kids,Movie,1,5.00,113
2,7669,"""Bungaku Shoujo"" Kyou no Oyatsu: Hatsukoi",Comedy,OVA,1,7.06,14351
3,7669,"""Bungaku Shoujo"" Kyou no Oyatsu: Hatsukoi",School,OVA,1,7.06,14351
4,7669,"""Bungaku Shoujo"" Kyou no Oyatsu: Hatsukoi",Fantasy,OVA,1,7.06,14351


### anime_stock_price.csv 파일
anime_stock_price.csv는 '도에이 애니메이션 주식회사'와 '주식회사 IG 포트'의 주가 데이터 이다. 
키워드 인수 parse_dates 에 열명을 리스트로 지정해서 로딩할 땡 datetime 형으로 취급한다. 

In [6]:
anime_stock_price_csv = './anime/anime_stock_price.csv'
pd.read_csv(anime_stock_price_csv, index_col=0,  parse_dates=['Date']).head()

,TOEI ANIMATION,IG Port
Date,,
2015-01-01,3356.86,1201.51
2015-01-02,3356.86,1201.51
2015-01-05,3396.12,1218.44
2015-01-06,3361.77,1201.51
2015-01-07,3297.97,1202.51
